In [1]:
# импортируем необходимые библиотеки
import pandas as pd

# импортируем класс TSDataset
from etna.datasets.tsdataset import TSDataset
# импортируем классы для выполнения преобразований
from etna.transforms import (
    StandardScalerTransform,
    MeanTransform, 
    LagTransform,
    FourierTransform,
    DateFlagsTransform,
    FilterFeaturesTransform, 
    OneHotEncoderTransform,
    TreeFeatureSelectionTransform
)
# импортируем класс Pipeline для 
# выполнения цепочки преобразований
from etna.pipeline import Pipeline
# импортируем класс SMAPE для оценки качества прогнозов
from etna.metrics import SMAPE

# импортируем RandomForestRegressor для вычисления 
# важностей признаков (понадобятся для отбора признаков)
from sklearn.ensemble import RandomForestRegressor

# импортируем функцию train_and_evaluate_model() для
# быстрой оценки качества модели и собственный класс
# LGBMMultiSegmentModel
from etna_utils import (train_and_evaluate_model,
                        LGBMMultiSegmentModel)

# отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

/Users/artemgruzdev/anaconda3/lib/python3.10/site-packages/etna/settings.py:61: UserWarning: `tsfresh` is not available, to install it, run `pip install tsfresh==0.19.0 && pip install protobuf==3.20.1`
  warnings.warn(


In [2]:
# загружаем исторический набор
df = pd.read_csv('Data/demand/train.csv', 
                 parse_dates=['date'])
# переименовываем date в timestamp, sales в target
df.rename(columns={'date': 'timestamp', 
                   'sales': 'target'}, inplace=True)
df

,timestamp,store,item,target
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [3]:
# сократим число рядов
df = df[df['store'].isin([1, 2])]
df

,timestamp,store,item,target
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
898387,2017-12-27,2,50,79
898388,2017-12-28,2,50,68
898389,2017-12-29,2,50,67
898390,2017-12-30,2,50,79


In [4]:
# загружаем набор новых данных
df_new = pd.read_csv('Data/demand/test.csv', 
                     parse_dates=['date'])
# переименовываем date в timestamp
df_new.rename(columns={'date': 'timestamp'}, inplace=True)
# сократим число рядов
df_new = df_new[df_new['store'].isin([1, 2])]
df_new

,id,timestamp,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44275,44275,2018-03-27,2,50
44276,44276,2018-03-28,2,50
44277,44277,2018-03-29,2,50
44278,44278,2018-03-30,2,50


In [5]:
# удалим идентификатор
df_new.drop('id', inplace=True, axis=1)
df_new

,timestamp,store,item
0,2018-01-01,1,1
1,2018-01-02,1,1
2,2018-01-03,1,1
3,2018-01-04,1,1
4,2018-01-05,1,1
...,...,...,...
44275,2018-03-27,2,50
44276,2018-03-28,2,50
44277,2018-03-29,2,50
44278,2018-03-30,2,50


In [6]:
# создаем сегменты - комбинации продуктовой 
# группы и номера магазина
df['segment'] = (df['store'].astype(str) + ' + ' 
                 + df['item'].astype(str))
df_new['segment'] = (df_new['store'].astype(str) + ' + ' 
                     + df_new['item'].astype(str))
df.head()

,timestamp,store,item,target,segment
0,2013-01-01,1,1,13,1 + 1
1,2013-01-02,1,1,11,1 + 1
2,2013-01-03,1,1,14,1 + 1
3,2013-01-04,1,1,13,1 + 1
4,2013-01-05,1,1,10,1 + 1


In [7]:
# присваиваем тип category столбцам store и item
for col in ['store', 'item']:
    df[col] = df[col].astype('category')
    df_new[col] = df_new[col].astype('category')

In [8]:
# формируем набор экзогенных переменных store и item
# для исторического периода
regressor_df = df[['timestamp', 'segment', 'store', 'item']].copy()
regressor_df

,timestamp,segment,store,item
0,2013-01-01,1 + 1,1,1
1,2013-01-02,1 + 1,1,1
2,2013-01-03,1 + 1,1,1
3,2013-01-04,1 + 1,1,1
4,2013-01-05,1 + 1,1,1
...,...,...,...,...
898387,2017-12-27,2 + 50,2,50
898388,2017-12-28,2 + 50,2,50
898389,2017-12-29,2 + 50,2,50
898390,2017-12-30,2 + 50,2,50


In [9]:
# формируем набор экзогенных переменных store и item
# для прогнозируемого периода
regressor_df_new = df_new.copy()
regressor_df_new

,timestamp,store,item,segment
0,2018-01-01,1,1,1 + 1
1,2018-01-02,1,1,1 + 1
2,2018-01-03,1,1,1 + 1
3,2018-01-04,1,1,1 + 1
4,2018-01-05,1,1,1 + 1
...,...,...,...,...
44275,2018-03-27,2,50,2 + 50
44276,2018-03-28,2,50,2 + 50
44277,2018-03-29,2,50,2 + 50
44278,2018-03-30,2,50,2 + 50


In [10]:
# сортируем признаки для последующей конкатенации
regressor_df = regressor_df.sort_index(
    axis=1, ascending=False)
regressor_df_new = regressor_df_new.sort_index(
    axis=1, ascending=False)

In [11]:
# конкатенируем набор с экзогенными переменными 
# для исторического периода и набор с экзогенными 
# переменными для прогнозируемого периода
regressor_df = pd.concat([regressor_df, regressor_df_new], axis=0)
# создаем новые экзогенные переменные
regressor_df['quarter'] = regressor_df['timestamp'].dt.quarter
regressor_df['quarter_start'] = regressor_df['timestamp'].dt.is_quarter_start
regressor_df['quarter_end'] = regressor_df['timestamp'].dt.is_quarter_end
regressor_df['month'] = regressor_df['timestamp'].dt.month_name()
regressor_df

,timestamp,store,segment,item,quarter,quarter_start,quarter_end,month
0,2013-01-01,1,1 + 1,1,1,True,False,January
1,2013-01-02,1,1 + 1,1,1,False,False,January
2,2013-01-03,1,1 + 1,1,1,False,False,January
3,2013-01-04,1,1 + 1,1,1,False,False,January
4,2013-01-05,1,1 + 1,1,1,False,False,January
...,...,...,...,...,...,...,...,...
44275,2018-03-27,2,2 + 50,50,1,False,False,March
44276,2018-03-28,2,2 + 50,50,1,False,False,March
44277,2018-03-29,2,2 + 50,50,1,False,False,March
44278,2018-03-30,2,2 + 50,50,1,False,False,March


In [12]:
# подготавливаем исторический набор эндогенных переменных
df.drop(['store', 'item'], axis=1, inplace=True)
df

,timestamp,target,segment
0,2013-01-01,13,1 + 1
1,2013-01-02,11,1 + 1
2,2013-01-03,14,1 + 1
3,2013-01-04,13,1 + 1
4,2013-01-05,10,1 + 1
...,...,...,...
898387,2017-12-27,79,2 + 50
898388,2017-12-28,68,2 + 50
898389,2017-12-29,67,2 + 50
898390,2017-12-30,79,2 + 50


In [13]:
# переводим исторический набор эндогенных 
# переменных в формат TSDataset
df = TSDataset.to_dataset(df)
df

segment,1 + 1,1 + 10,1 + 11,1 + 12,1 + 13,1 + 14,1 + 15,1 + 16,1 + 17,1 + 18,...,2 + 46,2 + 47,2 + 48,2 + 49,2 + 5,2 + 50,2 + 6,2 + 7,2 + 8,2 + 9
feature,target,target,target,target,target,target,target,target,target,target,...,target,target,target,target,target,target,target,target,target,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2013-01-01,13,37,37,33,37,22,42,14,13,38,...,43,12,45,29,4,43,34,39,49,28
2013-01-02,11,34,43,35,31,35,33,11,18,51,...,52,13,37,25,6,57,40,47,42,21
2013-01-03,14,32,34,41,50,26,45,12,15,42,...,30,5,45,30,12,45,41,43,46,29
2013-01-04,13,45,52,45,45,32,39,15,19,50,...,46,19,32,20,16,45,41,44,55,32
2013-01-05,10,35,45,46,49,31,47,22,16,56,...,41,18,53,20,9,55,47,55,64,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-27,14,55,43,47,49,42,62,16,29,54,...,59,22,48,30,20,79,56,65,79,51
2017-12-28,19,63,64,49,68,51,82,24,13,69,...,64,24,37,31,18,68,63,73,91,60


In [14]:
# переводим получившийся набор с экзогенными 
# переменными в формат TSDataset
regressor_df = TSDataset.to_dataset(regressor_df)
regressor_df

segment    1 + 1                                                  1 + 10  \
feature     item    month quarter quarter_end quarter_start store   item   
timestamp                                                                  
2013-01-01     1  January       1       False          True     1     10   
2013-01-02     1  January       1       False         False     1     10   
2013-01-03     1  January       1       False         False     1     10   
2013-01-04     1  January       1       False         False     1     10   
2013-01-05     1  January       1       False         False     1     10   
...          ...      ...     ...         ...           ...   ...    ...   
2018-03-27     1    March       1       False         False     1     10   
2018-03-28     1    March       1       False         False     1     10   
2018-03-29     1    March       1       False         False     1     10   
2018-03-30     1    March       1       False         False     1     10   
2018-03-31     1    March       1        True         False     1     10   

segment                                  ...   2 + 8              \
feature       month quarter quarter_end  ... quarter quarter_end   
timestamp                                ...                       
2013-01-01  January       1       False  ...       1       False   
2013-01-02  January       1       False  ...       1       False   
2013-01-03  January       1       False  ...       1       False   
2013-01-04  January       1       False  ...       1       False   
2013-01-05  January       1       False  ...       1       False   
...             ...     ...         ...  ...     ...         ...   
2018-03-27    March       1       False  ...       1       False   
2018-03-28    March       1       False  ...       1       False   
2018-03-29    March       1       False  ...       1       False   
2018-03-30    March       1       False  ...       1       False   
2018-03-31    March       1        True  ...       1        True   

segment                        2 + 9                               \
feature    quarter_start store  item    month quarter quarter_end   
timestamp                                                           
2013-01-01          True     2     9  January       1       False   
2013-01-02         False     2     9  January       1       False   
2013-01-03         False     2     9  January       1       False   
2013-01-04         False     2     9  January       1       False   
2013-01-05         False     2     9  January       1       False   
...                  ...   ...   ...      ...     ...         ...   
2018-03-27         False     2     9    March       1       False   
2018-03-28         False     2     9    March       1       False   
2018-03-29         False     2     9    March       1       False   
2018-03-30         False     2     9    March       1       False   
2018-03-31         False     2     9    March       1        True   

segment                         
feature    quarter_start store  
timestamp                       
2013-01-01          True     2  
2013-01-02         False     2  
2013-01-03         False     2  
2013-01-04         False     2  
2013-01-05         False     2  
...                  ...   ...  
2018-03-27         False     2  
2018-03-28         False     2  
2018-03-29         False     2  
2018-03-30         False     2  
2018-03-31         False     2  

[1916 rows x 600 columns]

In [15]:
# создаем объединенный набор для демонстрации
# преобразований/признаков без отбора признаков
# и применения в конвейерах
ts = TSDataset(df=df, freq='D', df_exog=regressor_df, 
               known_future='all')
# создаем объединенный набор для демонстрации
# преобразований/признаков с отбором признаков
select_ts = TSDataset(df=df, freq='D', df_exog=regressor_df, 
                      known_future='all')
ts

segment    1 + 1                                                           \
feature     item     month quarter quarter_end quarter_start store target   
timestamp                                                                   
2013-01-01     1   January       1       False          True     1   13.0   
2013-01-02     1   January       1       False         False     1   11.0   
2013-01-03     1   January       1       False         False     1   14.0   
2013-01-04     1   January       1       False         False     1   13.0   
2013-01-05     1   January       1       False         False     1   10.0   
...          ...       ...     ...         ...           ...   ...    ...   
2017-12-27     1  December       4       False         False     1   14.0   
2017-12-28     1  December       4       False         False     1   19.0   
2017-12-29     1  December       4       False         False     1   15.0   
2017-12-30     1  December       4       False         False     1   27.0   
2017-12-31     1  December       4        True         False     1   23.0   

segment    1 + 10                    ...         2 + 8              2 + 9  \
feature      item     month quarter  ... quarter_start store target  item   
timestamp                            ...                                    
2013-01-01     10   January       1  ...          True     2   49.0     9   
2013-01-02     10   January       1  ...         False     2   42.0     9   
2013-01-03     10   January       1  ...         False     2   46.0     9   
2013-01-04     10   January       1  ...         False     2   55.0     9   
2013-01-05     10   January       1  ...         False     2   64.0     9   
...           ...       ...     ...  ...           ...   ...    ...   ...   
2017-12-27     10  December       4  ...         False     2   79.0     9   
2017-12-28     10  December       4  ...         False     2   91.0     9   
2017-12-29     10  December       4  ...         False     2   99.0     9   
2017-12-30     10  December       4  ...         False     2  103.0     9   
2017-12-31     10  December       4  ...         False     2  124.0     9   

segment                                                              
feature        month quarter quarter_end quarter_start store target  
timestamp                                                            
2013-01-01   January       1       False          True     2   28.0  
2013-01-02   January       1       False         False     2   21.0  
2013-01-03   January       1       False         False     2   29.0  
2013-01-04   January       1       False         False     2   32.0  
2013-01-05   January       1       False         False     2   45.0  
...              ...     ...         ...           ...   ...    ...  
2017-12-27  December       4       False         False     2   51.0  
2017-12-28  December       4       False         False     2   60.0  
2017-12-29  December       4       False         False     2   48.0  
2017-12-30  December       4       False         False     2   58.0  
2017-12-31  December       4        True         False     2   75.0  

[1826 rows x 700 columns]

In [16]:
# создаем экземпляр класса StandardScalerTransform
scaler = StandardScalerTransform(in_column='target')

# создаем экземпляр класса LagTransform для генерации лагов
lags = LagTransform(in_column='target', 
                    lags=[i for i in range(90, 360, 20)],
                    out_column='lag')

# создаем экземпляр класса FourierTransform 
# для вычисления членов Фурье с периодом 365.24
# для моделирования годовой сезонности
fourier_year = FourierTransform(period=365.24, 
                                order=3, 
                                out_column='fourier_year')

# создаем экземпляры класса MeanTransform для 
# вычисления среднего по заданному окну
mean90 = MeanTransform(in_column='target', window=90, 
                        out_column='mean90')
mean180 = MeanTransform(in_column='target', window=180, 
                        out_column='mean180')
mean210 = MeanTransform(in_column='target', window=210, 
                        out_column='mean210')
mean240 = MeanTransform(in_column='target', window=240, 
                        out_column='mean240')
mean270 = MeanTransform(in_column='target', window=270, 
                        out_column='mean270')
mean360 = MeanTransform(in_column='target', window=360, 
                        out_column='mean360')

# создаем экземпляр класса DateFlagsTransform 
# для генерации признаков на основе дат
d_flags = DateFlagsTransform(day_number_in_year=True,
                             day_number_in_week=True,
                             day_number_in_month=True,
                             week_number_in_month=True,
                             week_number_in_year=True,
                             month_number_in_year=True,
                             season_number=True,
                             is_weekend=True,
                             out_column='datetime')

# создаем экземпляр класса OneHotEncoderTransform для
# создания дамми-переменных по уровням переменной month
one_hot = OneHotEncoderTransform(in_column='month', out_column='one_hot')

# создаем экземпляр класса FilterFeaturesTransform
# для удаления переменной month
drop = FilterFeaturesTransform(exclude=['month'])

# задаем список преобразований и признаков без отбора признаков
preprocess = [scaler, lags, fourier_year, mean90, mean180, mean210, 
              mean240, mean270, mean360, d_flags, one_hot, drop]

# задаем модель случайного леса для 
# вычисления важностей признаков
forest = RandomForestRegressor(max_depth=4, 
                               n_estimators=10, 
                               random_state=42)

# создаем экземпляр класса TreeFeatureSelectionTransform
# для отбора признаков с помощью важностей признаков,
# вычисленных моделью случайного леса
forest_select = TreeFeatureSelectionTransform(top_k=20, model=forest)

# задаем список преобразований и признаков с отбором признаков
preprocess_select = [scaler, lags, fourier_year, mean90, mean180, mean210, 
                     mean240, mean270, mean360, d_flags, one_hot, drop, 
                     forest_select]

In [17]:
# задаем горизонт прогнозирования
HORIZON = 90

# создаем экземпляр класса SMAPE
smape = SMAPE()

# создаем модель LGBMMultiSegmentModel
lgbm_model = LGBMMultiSegmentModel(n_estimators=150, n_jobs=1,
                                   learning_rate=0.1,
                                   num_leaves=10,
                                   min_data_in_leaf=120,
                                   subsample=0.8)

In [18]:
# создаем конвейер без отбора признаков
pipe = Pipeline(
    model=lgbm_model,
    transforms=preprocess,
    horizon=HORIZON)
        
# находим метрики моделей по сегментам 
# по итогам перекрестной проверки
metrics_df, _, _ = pipe.backtest(
    mode='expand', 
    n_folds=4,
    ts=ts, 
    metrics=[smape], 
    aggregate_metrics=True,
    joblib_params=dict(backend='loky'))

[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120


In [19]:
# смотрим метрики по первым 8 сегментам
metrics_df.head(8)

,segment,SMAPE
0,1 + 1,18.146527
1,1 + 10,10.469612
2,1 + 11,9.963219
3,1 + 12,10.010441
4,1 + 13,9.172918
5,1 + 14,10.763307
6,1 + 15,9.159376
7,1 + 16,17.314056


In [20]:
# смотрим значение SMAPE, усредненное по сегментам
metric = metrics_df['SMAPE'].mean()
print(f'mean SMAPE: {metric:.4f}')

mean SMAPE: 11.7264


In [21]:
# создаем конвейер с отбором признаков
pipe_select = Pipeline(
    model=lgbm_model,
    transforms=preprocess_select,
    horizon=HORIZON)
        
# находим метрики моделей по сегментам 
# по итогам перекрестной проверки
metrics_df, _, _ = pipe_select.backtest(
    mode='expand', 
    n_folds=4,
    ts=ts, 
    metrics=[smape], 
    aggregate_metrics=True,
    joblib_params=dict(backend='loky'))

[LightGBM] [Warning] min_data_in_leaf is set=120, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=120


In [22]:
# смотрим метрики по первым 8 сегментам
metrics_df.head(8)

,segment,SMAPE
0,1 + 1,18.420440
1,1 + 10,10.216706
2,1 + 11,9.709312
3,1 + 12,9.994314
4,1 + 13,8.752091
5,1 + 14,10.496183
6,1 + 15,9.116431
7,1 + 16,17.423713


In [23]:
# смотрим значение SMAPE, усредненное по сегментам
metric = metrics_df['SMAPE'].mean()
print(f'mean SMAPE: {metric:.4f}')

mean SMAPE: 11.5778


In [24]:
# применяем преобразования/создаем признаки без 
# отбора признаков на всем историческом наборе
ts.fit_transform(preprocess)
ts

segment                           1 + 1                              \
feature    datetime_day_number_in_month datetime_day_number_in_week   
timestamp                                                             
2013-01-01                            1                           1   
2013-01-02                            2                           2   
2013-01-03                            3                           3   
2013-01-04                            4                           4   
2013-01-05                            5                           5   
...                                 ...                         ...   
2017-12-27                           27                           2   
2017-12-28                           28                           3   
2017-12-29                           29                           4   
2017-12-30                           30                           5   
2017-12-31                           31                           6   

segment                                                     \
feature    datetime_day_number_in_year datetime_is_weekend   
timestamp                                                    
2013-01-01                           1               False   
2013-01-02                           2               False   
2013-01-03                           3               False   
2013-01-04                           4               False   
2013-01-05                           5                True   
...                                ...                 ...   
2017-12-27                         362               False   
2017-12-28                         363               False   
2017-12-29                         364               False   
2017-12-30                         365                True   
2017-12-31                         366                True   

segment                                                          \
feature    datetime_month_number_in_year datetime_season_number   
timestamp                                                         
2013-01-01                             1                      1   
2013-01-02                             1                      1   
2013-01-03                             1                      1   
2013-01-04                             1                      1   
2013-01-05                             1                      1   
...                                  ...                    ...   
2017-12-27                            12                      1   
2017-12-28                            12                      1   
2017-12-29                            12                      1   
2017-12-30                            12                      1   
2017-12-31                            12                      1   

segment                                                                \
feature    datetime_week_number_in_month datetime_week_number_in_year   
timestamp                                                               
2013-01-01                             1                            1   
2013-01-02                             1                            1   
2013-01-03                             1                            1   
2013-01-04                             1                            1   
2013-01-05                             1                            1   
...                                  ...                          ...   
2017-12-27                             5                           52   
2017-12-28                             5                           52   
2017-12-29                             5                           52   
2017-12-30                             5                           52   
2017-12-31                             5                           52   

segment                                   ...     2 + 9                      \
feature    fourier_year_1 fourier_year_2  ... one_hot_5 one_hot_6 one_hot_7   
timestamp       

In [25]:
# смотрим признаки и количество признаков
features = ts.columns.get_level_values(1).difference(
    ['target']).unique().tolist()
print(features)
print('')
print(f'количество признаков: {len(features)}')

['datetime_day_number_in_month', 'datetime_day_number_in_week', 'datetime_day_number_in_year', 'datetime_is_weekend', 'datetime_month_number_in_year', 'datetime_season_number', 'datetime_week_number_in_month', 'datetime_week_number_in_year', 'fourier_year_1', 'fourier_year_2', 'fourier_year_3', 'fourier_year_4', 'fourier_year_5', 'fourier_year_6', 'item', 'lag_110', 'lag_130', 'lag_150', 'lag_170', 'lag_190', 'lag_210', 'lag_230', 'lag_250', 'lag_270', 'lag_290', 'lag_310', 'lag_330', 'lag_350', 'lag_90', 'mean180', 'mean210', 'mean240', 'mean270', 'mean360', 'mean90', 'one_hot_0', 'one_hot_1', 'one_hot_10', 'one_hot_11', 'one_hot_2', 'one_hot_3', 'one_hot_4', 'one_hot_5', 'one_hot_6', 'one_hot_7', 'one_hot_8', 'one_hot_9', 'quarter', 'quarter_end', 'quarter_start', 'store']

количество признаков: 51


In [26]:
# применяем преобразования/создаем признаки c
# отбором признаков на всем историческом наборе
select_ts.fit_transform(preprocess_select)
select_ts

segment                          1 + 1                              \
feature    datetime_day_number_in_week datetime_day_number_in_year   
timestamp                                                            
2013-01-01                           1                           1   
2013-01-02                           2                           2   
2013-01-03                           3                           3   
2013-01-04                           4                           4   
2013-01-05                           5                           5   
...                                ...                         ...   
2017-12-27                           2                         362   
2017-12-28                           3                         363   
2017-12-29                           4                         364   
2017-12-30                           5                         365   
2017-12-31                           6                         366   

segment                                                               \
feature    datetime_is_weekend datetime_season_number fourier_year_2   
timestamp                                                              
2013-01-01               False                      1       1.000000   
2013-01-02               False                      1       0.999852   
2013-01-03               False                      1       0.999408   
2013-01-04               False                      1       0.998669   
2013-01-05                True                      1       0.997633   
...                        ...                    ...            ...   
2017-12-27               False                      1       0.996002   
2017-12-28               False                      1       0.997391   
2017-12-29               False                      1       0.998485   
2017-12-30                True                      1       0.999284   
2017-12-31                True                      1       0.999787   

segment                                                                 ...  \
feature    fourier_year_4 fourier_year_5 fourier_year_6 item   lag_110  ...   
timestamp                                                               ...   
2013-01-01       1.000000       0.000000       1.000000    1       NaN  ...   
2013-01-02       0.999408       0.051586       0.998669    1       NaN  ...   
2013-01-03       0.997633       0.103034       0.994678    1       NaN  ...   
2013-01-04       0.994678       0.154208       0.988038    1       NaN  ...   
2013-01-05       0.990545       0.204972       0.978768    1       NaN  ...   
...                   ...            ...            ...  ...       ...  ...   
2017-12-27       0.984038      -0.265155       0.964206    1  1.191314  ...   
2017-12-28       0.989577      -0.215063       0.976600    1  1.191314  ...   
2017-12-29       0.993945      -0.164398       0.986394    1  1.488087  ...   
2017-12-30       0.997137      -0.113295       0.993561    1 -0.589319  ...   
2017-12-31       0.999148      -0.061891       0.998083    1  0.004226  ...   

segment        2 + 9                                                    \
feature      lag_150   lag_170   lag_190   lag_210   lag_230   lag_250   
timestamp                                                                
2013-01-01       NaN       NaN       NaN       NaN       NaN       NaN   
2013-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2013-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2013-01-04       NaN       NaN       NaN       NaN       NaN       NaN   
2013-01-05       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2017-12-27  2.887960  0.248641  1.293371  0.578556  1.898215  1.073428   
2017-12-28  0.083683  1.293371  0.908471  0.908471  0.963456  0.633542   
2017-12-29  0.853485  1.073428  0.963456  1.788244  1.953201  1.788244   
2017-12-30  0.0

In [27]:
# смотрим отобранные признаки и количество отобранных признаков
select_features = select_ts.columns.get_level_values(1).difference(
    ['target']).unique().tolist()
print(select_features)
print('')
print(f'количество отобранных признаков: {len(select_features)}')

['datetime_day_number_in_week', 'datetime_day_number_in_year', 'datetime_is_weekend', 'datetime_season_number', 'fourier_year_2', 'fourier_year_4', 'fourier_year_5', 'fourier_year_6', 'item', 'lag_110', 'lag_130', 'lag_150', 'lag_170', 'lag_190', 'lag_210', 'lag_230', 'lag_250', 'lag_290', 'mean360', 'store']

количество отобранных признаков: 20
